In [1]:
import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from pydataxm import *                           #Se realiza la importación de las librerias necesarias para ejecutar                        
from pydataxm.pydataxm import ReadDB as apiXM 
import datetime as dt                            
import pandas as pd
from pathlib import Path
import os

from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
# Información del proyecto y autenticación a BQ
project_id = "enersinc-tbsg-bq"
key_path = "C:\BigQuery\eramirez-tbsg.json"

# Cargar las credenciales del archivo JSON
credentials = service_account.Credentials.from_service_account_file(key_path)

# Crear el cliente de BigQuery
client = bigquery.Client(project=project_id, credentials=credentials)

In [3]:
s_path=Path.cwd()
s_prt_path=s_path.parent

objetoAPI = pydataxm.ReadDB()     

df =objetoAPI.get_collections()  

# Get main path and other folders
s_mainpath=Path.cwd()
sRutaPrint=s_mainpath.joinpath('Print')

In [4]:
FechaIni=dt.date(2025,8,25)
FechaFin=dt.date(2025,9, 25)

In [5]:
#Leer archivo de mapeos
spathFile=r'C:\Alejo\ExtracciónDatos\Parametros.xlsx'
df_plt_sub=pd.read_excel(spathFile,sheet_name='Menores')
df_plt_sub=pd.concat([df_plt_sub,pd.read_excel(spathFile,sheet_name='DC')],axis=0)

In [6]:
# Consulta a la maestra de recursos
query = """
select distinct *
from tbsg.public_maestra_recurso 
where recurso_despacho is not null
"""
# Ejecutar la consulta
df_MaestraRec= client.query(query).to_dataframe()

In [7]:
df_RecIni= apiXM.request_data(pydataxm.ReadDB(),    #Se indica el objeto que contiene el serivicio
                        "ListadoRecursos",                #Se indica el nombre de la métrica tal como se llama en el campo metricID
                        "Sistema",                 #Campo que indica si es Sistema, Agente, Recurso, Comercializador
                        FechaIni,       #Corresponde a la fecha inicial de la consulta
                        FechaFin)      #Corresponde a la fecha final de la consulta

# Units kWh, Descarga de la disponibilidad real
df_DispDecIni= apiXM.request_data(pydataxm.ReadDB(),    #Se indica el objeto que contiene el serivicio
                        "DispoDeclarada",                #Se indica el nombre de la métrica tal como se llama en el campo metricID
                        "Recurso",                 #Campo que indica si es Sistema, Agente, Recurso, Comercializador
                        FechaIni,       #Corresponde a la fecha inicial de la consulta
                        FechaFin)      #Corresponde a la fecha final de la consulta

# Units kWh, Descarga de la disponibilidad real
df_CENIni= apiXM.request_data(pydataxm.ReadDB(),    #Se indica el objeto que contiene el serivicio
                        "CapEfecNeta",                #Se indica el nombre de la métrica tal como se llama en el campo metricID
                        "Recurso",                 #Campo que indica si es Sistema, Agente, Recurso, Comercializador
                        FechaIni,       #Corresponde a la fecha inicial de la consulta
                        FechaFin)      #Corresponde a la fecha final de la consulta

In [8]:
df_CEN=df_CENIni.copy()
df_Rec=df_RecIni.copy()
df_DispDec=df_DispDecIni.copy()

# df_CEN.fillna(0, inplace=True)
df_CEN['Value']=df_CEN['Value']/1000
df_CEN.rename(columns={'Value': 'CEN'}, inplace=True)
# df_CEN=df_CEN[df_CEN.Date=='2024-09-01']
df_DispDec.fillna(0, inplace=True)

if 'Date' in df_Rec.columns:
    df_Rec=df_Rec.drop('Date',axis=1)

# if 'Date' in df_CEN.columns:
#     df_CEN=df_CEN.drop('Date',axis=1)

l_per=[]
for i in range(1,25):
    if i<10:
        df_DispDec=df_DispDec.rename(columns={'Values_Hour0' + str(i) : str(i) })
    else:
        df_DispDec=df_DispDec.rename(columns={'Values_Hour' + str(i) : str(i) })
    df_DispDec[str(i)]=df_DispDec[str(i)]/1000
    l_per.append(str(i))


In [9]:
l_col=['Date','Values_code','Values_Name', 'Values_Type', 'Values_Disp','Values_RecType', 'Values_State',
       '1', '2', '3','4' ,'5', '6', '7','8','9', '10', '11','12', '13', '14', '15','16', '17', '18', '19','20', '21', '22', '23','24']

df_Disp_Rec=df_DispDec.merge(df_Rec,left_on=['Values_code'],right_on=['Values_Code'], how='left')[l_col]
df_Disp_Rec=df_Disp_Rec.groupby(['Date','Values_code','Values_Name', 'Values_Type', 'Values_Disp','Values_RecType', 'Values_State'])[l_per].median().reset_index()

l_col.append('CEN')
df_DispRec_CEN=df_Disp_Rec.merge(df_CEN,left_on=['Values_code','Date'],right_on=['Code','Date'], how='left')[l_col]

df_DispRec_CEN['CEN'] = df_DispRec_CEN['CEN'].fillna(df_DispRec_CEN[l_per].max(axis=1))

l_col.append('Subarea')


# # Fill missing CEN values with df_menores dataframe where Values_Name matches with Planta
# df_DispRec_CEN = df_DispRec_CEN.merge(df_menores[['Planta','Subarea']], left_on='Values_Name', right_on='Planta', how='left')


df_DispRec_CEN = df_DispRec_CEN.merge(df_plt_sub[['Planta','Subarea']], left_on='Values_Name', right_on='Planta', how='left')
l_col.append('subarea')
df_MaestraRec = df_MaestraRec.drop_duplicates(subset='codsic_planta')
df_DispRec_CEN=df_DispRec_CEN.merge(df_MaestraRec[['codsic_planta','subarea']],left_on=['Values_code'],right_on=['codsic_planta'], how='left')[l_col]
df_DispRec_CEN['Subarea'] = df_DispRec_CEN['Subarea'].fillna(df_DispRec_CEN['subarea'])
df_DispRec_CEN['Subarea'] = df_DispRec_CEN['Subarea'].fillna('No tiene')
df_DispRec_CEN['Subarea'] = df_DispRec_CEN['Subarea'].replace({
       'SubArea Cordoba_Sucre':'CORDOSUC',
       'SubArea Atlantico':'ATLANTIC',
       'SubArea Cerromatoso':'CERROMAT',
       'SubArea GCM':'GCM',
       'SubArea Bolivar':'BOLIVAR'
})
df_DispRec_CEN['subareafinal'] = df_DispRec_CEN['Subarea'].apply(lambda x: x if x in ['ATLANTIC','BOLIVAR','GCM','CERROMAT','CORDOSUC'] else 'INTERIOR')
df_DispRec_CEN.drop(columns=['Subarea','subarea'], inplace=True)
df_CENFin=df_DispRec_CEN.groupby(['Values_code'])['CEN'].max().reset_index()

df_DispRec_CEN.to_csv(sRutaPrint.joinpath('Disponibiliad.csv'))



In [10]:
# Apply the filter to discard rows where both conditions do not match simultaneously
df_Variable = df_DispRec_CEN[~((df_DispRec_CEN['Values_Type'] == 'HIDRAULICA') & (df_DispRec_CEN['Values_Disp'] == 'DESPACHADO CENTRALMENTE')) & 
                             ~((df_DispRec_CEN['Values_Type'] == 'TERMICA') & (df_DispRec_CEN['Values_Disp'] == 'DESPACHADO CENTRALMENTE'))]
# Display the filtered dataframe
df_Variable.to_csv(sRutaPrint.joinpath('DisponibiliadVar.csv'))

In [11]:
# Obtener información de las plantas despachadas centralmente que tienen peso 
df_VariableDC=df_Variable[(df_Variable['Values_Name'].isin(['LATAM SOLAR LA LOMA','EL PASO','GUAYEPO','FUNDACION','CARACOLI I','PARQUE SOLAR LA UNION']))]
df_VariableDC=df_VariableDC.groupby(['Values_Name','Values_Type','Values_Disp','Values_RecType','subareafinal'])[l_per].mean().reset_index()
for col in l_per:
    df_VariableDC[col] = df_VariableDC[col].round(0)
df_VariableDC.to_csv(sRutaPrint.joinpath('DisponibiliadVarDC.csv'))

In [12]:
# Obtener información de las plantas despachadas centralmente que no tienen peso 
df_VariableOtros=df_Variable[(~df_Variable['Values_Name'].isin(['LATAM SOLAR LA LOMA','EL PASO','GUAYEPO','FUNDACION','CARACOLI I','PARQUE SOLAR LA UNION']))]
df_VariableOtros['Values_Type'] = df_VariableOtros['Values_Type'].apply(lambda x: x if x in ['EOLICA','SOLAR'] else 'NDC')
df_VariableOtros.loc[df_VariableOtros['Values_Disp'] == 'NO DESPACHADO CENTRALMENTE', 'Values_Type'] = 'NDC'
df_VariableOtros=df_VariableOtros.groupby(['Values_Name','Values_Type','Values_Disp','subareafinal'])[l_per].mean().reset_index()
df_VariableOtros=df_VariableOtros.groupby(['Values_Type','Values_Disp','subareafinal'])[l_per].sum().reset_index()

for col in l_per:
    df_VariableOtros[col] = df_VariableOtros[col].round(0)
df_VariableOtros.to_csv(sRutaPrint.joinpath('DisponibiliadVarOtros.csv'))

In [15]:
# Units kWh, Descarga de la disponibilidad real
df_GenRealIni= apiXM.request_data(pydataxm.ReadDB(),    #Se indica el objeto que contiene el serivicio
                        "Gene",                #Se indica el nombre de la métrica tal como se llama en el campo metricID
                        "Recurso",                 #Campo que indica si es Sistema, Agente, Recurso, Comercializador
                        FechaIni,       #Corresponde a la fecha inicial de la consulta
                        FechaFin)      #Corresponde a la fecha final de la consulta
df_GenRealIni['Values_code']=df_GenRealIni['Values_code'].astype(str)

In [16]:
df_GenReal=df_GenRealIni.copy()
df_GenReal.fillna(0, inplace=True)

l_per=[]
for i in range(1,25):
    if i<10:
        df_GenReal=df_GenReal.rename(columns={'Values_Hour0' + str(i) : str(i) })
    else:
        df_GenReal=df_GenReal.rename(columns={'Values_Hour' + str(i) : str(i) })
    df_GenReal[str(i)]=df_GenReal[str(i)]/1000
    l_per.append(str(i))


In [17]:
l_perMod=l_per.copy()
l_perMod.append('CEN')
l_col=['Date','Values_code','Values_Name', 'Values_Type', 'Values_Disp','Values_RecType', 'Values_State',
       '1', '2', '3','4' ,'5', '6', '7','8','9', '10', '11','12', '13', '14', '15','16', '17', '18', '19','20', '21', '22', '23','24']

df_Gen_Rec=df_GenReal.merge(df_Rec,left_on=['Values_code'],right_on=['Values_Code'], how='left')[l_col]
l_col.append('CEN')
df_GenRec_CEN=df_Gen_Rec.merge(df_CENFin,left_on=['Values_code'],right_on=['Values_code'], how='left')[l_col]
df_GenRec_CEN['Mes']=df_GenRec_CEN['Date'].dt.month
df_GenRec_CEN=df_GenRec_CEN[(df_GenRec_CEN.Values_Disp=='NO DESPACHADO CENTRALMENTE')]
df_GenRec_CEN.to_csv('GenNDC_CEN.csv')
df_GenRec_CEN=df_GenRec_CEN.groupby(['Date','Mes','Values_Type', 'Values_Disp'])[l_perMod].sum()
df_GenRec_CEN=df_GenRec_CEN.reset_index()
df_GenRec_CEN=df_GenRec_CEN.groupby(['Mes','Values_Type', 'Values_Disp'])[l_perMod].median()
df_GenRec_CEN=df_GenRec_CEN.reset_index()
# df_GenRec_CEN=df_GenRec_CEN.groupby(['Mes','Values_Disp'])[l_per].sum()
df_GenRec_CEN.to_csv(sRutaPrint.joinpath('GenNDC.csv'))
# df_GenRec_CEN=df_GenRec_CEN[(df_GenRec_CEN.Mes==9)]
# df_GenRec_CEN.head(15)


In [18]:
df_Facors=df_GenRec_CEN.copy()

for i in range(1,25):
    df_Facors[str(i)]=df_Facors[str(i)]/df_Facors['CEN']

df_Facors.to_csv(sRutaPrint.joinpath('FactoresNDC.csv'))
df_Facors.head(10)

,Mes,Values_Type,Values_Disp,1,2,3,4,5,6,7,...,16,17,18,19,20,21,22,23,24,CEN
0,5,COGENERADOR,NO DESPACHADO CENTRALMENTE,0.455195,0.446775,0.458547,0.465045,0.458890,0.486160,0.465445,...,0.434820,0.431618,0.447033,0.444684,0.448478,0.445486,0.447718,0.434439,0.453079,129.6500
1,5,EOLICA,NO DESPACHADO CENTRALMENTE,0.353039,0.301541,0.314229,0.262602,0.345435,0.367935,0.361354,...,1.018041,1.009525,0.959254,0.880904,0.823748,0.652812,0.511692,0.525022,0.451933,22.7800
2,5,HIDRAULICA,NO DESPACHADO CENTRALMENTE,0.745198,0.742652,0.741184,0.733197,0.744046,0.744956,0.734145,...,0.766973,0.768030,0.771488,0.782192,0.776060,0.784616,0.772610,0.769293,0.764498,863.1820
3,5,SOLAR,NO DESPACHADO CENTRALMENTE,0.000000,0.000000,0.000000,0.000000,0.000000,0.002091,0.081874,...,0.536984,0.332467,0.083072,0.001223,0.000000,0.000000,0.000000,0.000000,0.000000,688.6853
4,5,TERMICA,NO DESPACHADO CENTRALMENTE,0.533775,0.533074,0.534130,0.529269,0.535748,0.543174,0.547807,...,0.550307,0.520180,0.514748,0.516378,0.527494,0.540771,0.552473,0.557525,0.549064,167.8850
